In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

# **Static, partial equilibrium model**
### **Production module, example 3:**

*Example 3 defines a partial equilibrium model from data, including nesting structure, and calibration targets. The model is as stylized as possible, ignoring investments, trade and taxes.*

**NB: The module can run with the cobb-douglas specification ($\sigma=1$). However, when this is the case, the sum of $\mu$-parameters should equal 1 (to keep constant returns to scale).**

### **1: Read in data, and nesting structure**

Read in general equilibrium data:

In [2]:
data_folder = os.getcwd()+'\\PE_P\\Example_3'
data = {'Production_v': data_folder+'\\Production_v.xlsx', 'Production_p': data_folder+'\\Production_p.xlsx'}
components = ['domestic']
db_GE = ReadData.read_data.main(data,components=components)

Extract data on sector 's1' from GE data:

In [3]:
s0 = 's1'
db_PE = ReadData.PE_from_GE(db_GE,s0)

Read in nesting:

In [4]:
nt = nesting_tree.nesting_tree(name='PE_Example3')

*Input-part:*

In [5]:
nt.add_tree(data_folder+'\\nest_s1_in.xlsx',name='s1_in')
read_type = {'1dvars': {'sheets': ['sigma'],'names':{}}, 'vars_panel': {'sheets': {'mu': 2},'names': {}}}
nt.trees['s1_in'].database.read_from_excel(data_folder+'\\nest_s1_in.xlsx',read_type)

*Output-split:*

In [6]:
nt.add_tree(data_folder+'\\nest_s1_out.xlsx',name='s1_out',**{'type_io': 'output', 'type_f': 'CET'})
nt.trees['s1_out'].database.read_from_excel(data_folder+'\\nest_s1_out.xlsx',read_type)

*NB: Note that both Y1,Y2 are used as inputs and outputs. To accomodate for this, we use the feature of temporarily applying new set-names to selected elements of the nesting tree. A dict keeps track of the original names and reverts the names after collecting the nesting trees later on. This feature is called by adding 'temp_namespace' as in the following:*

In [7]:
nt.trees['s1_out'].temp_namespace = {'Y1': 'Y1_out', 'Y2': 'Y2_out'}

### **2: Set up baseline model**

*Compile tree attributes:*

*Note that once the aggregate tree has been compiled, the temporary namespace used to have Y1,Y2 as both inputs and outputs are reversed per default - hence the 'temporary' in temporary namespace. If this is not the case, it should be specified through other means than the 'self.temp_namspace' attribute.*

In [8]:
nt.run_all()

*Initiate production module with calibration data:*

In [9]:
pm = PE.GPM_STA_PE.production(nt,**{'calib_db': db_PE, 'work_folder': work_folder, 'data_folder': data_folder})

*Run calibration function. This works as follows:*
1. Runs baseline model:
    * Initialize variables (if no levels are given, some default values are provided).
    * Write gams-code to the folder self.data_folder (specified in the options above).
    * Create a model-instance named 'baseline' per default (can be specified with kwargs 'name_base': name). 
    * Create a checkpoint named 'based 'baseline', and solve model.
2. Create and ready calibration model instance:
    * Create model-instance named 'calib' per default (can be specified with kwargs 'name_calib': name).
    * Endogenize/exogenize relevant variables for calibration: self.calib contains gams-code for endogenizing $\mu$-parameters, and exogenizing prices/quantities. Note: Initially, the calibration data for quantities/prices are not applied; this is to check that the endogenizing/exogenizing does not break the 'squareness' of the model.
3. Run calibration model:
    * The default mode is to calibrate the model in a 'sneaky' manner: The solve_sneakily method creates a database with grids of data between the initial solution, and the data provided in self.calib_db. We then gradually update exogenous variables in 'n_steps' towards the calibration targets.
    * Note: When calibrating the standard partial equilibrium model, sneaking up on the solution is not as straightforward: If the production sector has constant returns to scale, it has to hold that the value of inputs equals the value of outputs; calibration of $\mu$-parameters cannot alter this fact. Thus, making linear grids between the initial solution and the calibration target can result in unbalanced data (making the model potentially infeasible to solve for). To remedy this, the solve_sneakily method allow the user to apply one of three approaches that guarantees balanced data: 
        * For $type\_=$'both': Input prices and quantities are linearly spaced. Let $(p0,pT,q0,qT)$ denote the initial and final (last step of the sneaky-solve) price/quantity vectors for outputs in the sector. Let $i$ index the outputs quantities/prices, and $j$ the intermediate step in the sneaky-solve. We then solve:
        $$\begin{align}
            \sum_i (x_jpT(i)+(1-x_j)p0(i))*(x_jqT(i)+(1-x_j)q0(i)) = TC_j,
        \end{align}$$
        where $TC_j$ is the total costs in the j'th step (defined by summing over inputs). That is, $x_j$ solved for the weight on the final step $T$. 
        * For $type\_=$'price': Input prices and quantities are linearly spaced. Output prices are linearly spaced as well. Quantities are spaced by solving for $x_j$:
        $$\begin{align}
            \sum_i p_j(i)*(x_jqT(i)+(1-x_j)q0(i)) = TC_j.
        \end{align}$$
        Thus output quantities are non-linearly spaced to ensure that the step is feasible.
        * For $type\_=$'quant': The reverse case of 'price'; output quantities are linearly spaced, output prices are non-linearly spaced to ensure feasibility.
    * Also note: Besides choosing different versions of creating balanced data, the module has been updated to create different types of nonlinear grids. Thus gridtype can be specified as $\lbrace pol,rust,linear\rbrace$. The 'linear' option simply applies the 'np.linspace' method. 'pol' and 'rust' methods specify grids as follows:
    $$\begin{align}
        \text{Polynomial (pol):} && x_i &= x_0+(x_N-x_0)\left(\dfrac{i-1}{N}\right)^{\phi} \\ 
        \text{Rust-spaced (rust):} && x_i &= x_{i-1}+\dfrac{x_N-x_{i-1}}{(N-i+1)^{\phi}}.
    \end{align}$$
    In both cases, a $\phi=1$ implies a linear grid, $\phi<1$ places more gridpoints close to the end of the grid $(x_N)$, and vice versa for $\phi>1$. 

*Here we calibrate by taking a loooooot of steps, to make sure to get it right (this adds the polynomially spaced grid with weights in the end of the grid):*

In [10]:
pm.calibrate(type_='price',kwargs_shock={'n_steps': 10,'gridtype':'pol','phi': 0.5})

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\GamsPythonModels\work_folder\_gams_py_gjo1.lst for more details

*If succesfull, store calibrated data:*

In [11]:
if pm.model_instances['calib'].modelstat == 16 and pm.model_instances['calib'].solvestat==1:
    pm.model_instances['calib'].out_db.db_Gdx.export(pm.data_folder+'\\calibrated.gdx')

## How to pickle a model instance:

In [12]:
pm.data_folder

'C:\\Users\\sxj477\\Documents\\GitHub\\GamsPythonModels\\Examples\\PE_P\\Example_3'

In [13]:
pm.model_instances['calib'].export(pm.data_folder,'PE_Production_3_calib')